# Tracking FII flows

Plots the Foreign Institutional Investment flows in/out of Indian capital markets.
Follows the documentation [here](https://github.com/shyams80/plutons/blob/master/docs-R/InvestmentFlowsIndia.ipynb)

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

#initialize
iflows <- InvestmentFlowsIndia()

In [ ]:
fiiDebt <- iflows$FiiCashMarket() %>%
    filter(SECURITY_TYPE == 'Debt') %>%
    collect() %>%
    mutate(Y=year(TIME_STAMP), M=month(TIME_STAMP)) %>%
    group_by(Y, M) %>%
    summarize(NET = sum(BUY_VALUE - SELL_VALUE)) %>%
    ungroup() %>%
    arrange(Y, M) %>%
    mutate(YM = paste0(as.character(Y), str_pad(M, 2, pad="0"))) %>%
    select(YM, NET) %>%
    as.data.frame()

### plot the flows

In [ ]:
plotFiiFlows <- function(fii, mainTitle){
    xAxisTicks <- fiiDebt[seq(from=1, to=nrow(fii), length.out=20), 'YM']
    firstDate <- first(fii$YM)
    lastDate <- last(fii$YM)

    ggplot(fii, aes(x=YM, y=NET)) +
        theme_economist() +
        theme(axis.text.x=element_text(angle=90, hjust=1)) +
        geom_bar(stat="identity", position=position_dodge()) +
        scale_x_discrete(breaks = xAxisTicks) +
        #geom_text_repel(aes(label= round(NET, 2)), position = position_dodge(0.9)) +
        labs(x='', y='Rs. Crore', color='', title=plotFiiFlows, 
             subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
        annotate("text", x=1, y=min(fii$NET, na.rm=T), label = "@StockViz", 
                 hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)
    
    print(summary(fii$NET))
}

In [ ]:
plotFiiFlows(fiiDebt, "FII Debt Flows")

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)